1. Read json file
2. Set up the sampler through batchrunner
3. generate the input file
4. save it.

In [13]:
# Import relevant libraries
import os
from ModularCirc import BatchRunner
import numpy as np
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT
from SALib.analyze.sobol import analyze
from SALib.sample.sobol import sample as sobol_sample
from SALib.analyze.sobol import analyze as sobol_analyze
import matplotlib.pyplot as plt
from autoemulate.compare import AutoEmulate
from SALib.analyze.sobol import analyze
import numpy as np
import json

In [14]:
# Define variables
param_path = "./parameters_naghavi_constrained_fixed_T_v_tot.json"
# param_path = "./parameters_naghavi.json"

# Get the filename from the path, without extension
param_filename = os.path.splitext(os.path.basename(param_path))[0]

n_samples = 2**15 # 2^11 = 2048 samples

simulation_out_path = f'outputs/simulations/output_{n_samples}_samples_{param_filename}/'

os.makedirs(os.path.join(simulation_out_path, 
                         f'raw_simulations_{n_samples}_samples'), 
                         exist_ok=True)
## read in save parameters to the folder where simulations r saved.
with open(param_path, 'r') as f: 
    params = json.load(f)
# save parameters to the folder where simulations are saved.
with open(os.path.join(simulation_out_path, 'parameters.json'), 'w') as f:
    json.dump(params, f, indent=4)

In [15]:
# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here? 
br.setup_sampler(param_path)
br.sample(n_samples)

In [16]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.delay,la.t_tr,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max,T,v_tot
0,305.576111,0.364705,1047.778645,2.796825,7.379755,116.738799,5.767604,3.872886,0.253016,0.650535,...,150,225,25,150,0,420,25,280,800,5000
1,239.201979,0.294212,1290.672647,3.257753,10.025982,101.463371,6.617964,4.674997,0.357100,0.503905,...,150,225,25,150,0,420,25,280,800,5000
2,218.335616,0.324832,910.186916,2.417528,10.382382,94.085593,5.249952,4.504283,0.334988,0.421357,...,150,225,25,150,0,420,25,280,800,5000
3,277.901886,0.254335,1145.884560,3.622402,8.282241,137.229613,7.212086,3.251764,0.293604,0.550802,...,150,225,25,150,0,420,25,280,800,5000
4,273.455547,0.305361,1370.442680,3.441763,8.862728,123.493007,4.754028,4.111263,0.314531,0.522093,...,150,225,25,150,0,420,25,280,800,5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32763,273.449653,0.260270,1145.757946,2.614628,10.700671,104.489212,5.947154,4.092908,0.233738,0.448196,...,150,225,25,150,0,420,25,280,800,5000
32764,277.907881,0.304119,1369.509320,2.434000,10.400068,90.731860,6.114197,4.936055,0.358119,0.626810,...,150,225,25,150,0,420,25,280,800,5000
32765,218.341541,0.243436,1109.569083,3.619842,8.264705,134.332289,5.451334,3.683882,0.251705,0.497369,...,150,225,25,150,0,420,25,280,800,5000
32766,239.196018,0.339278,1225.251313,2.804519,7.362138,126.717609,7.059604,3.537223,0.294924,0.432007,...,150,225,25,150,0,420,25,280,800,5000


In [17]:
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}
br.map_sample_timings(
    ref_time=1000., # double check if 1000 or 1
    map=map_
    )

In [18]:
br.map_vessel_volume()

In [19]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max,T,ao.v,art.v,ven.v
0,305.576111,0.364705,1047.778645,2.796825,7.379755,116.738799,5.767604,3.872886,0.253016,0.650535,...,20.0,120.0,0,336.0,20.0,224.0,800,15.208670,116.631258,4868.160073
1,239.201979,0.294212,1290.672647,3.257753,10.025982,101.463371,6.617964,4.674997,0.357100,0.503905,...,20.0,120.0,0,336.0,20.0,224.0,800,14.008070,155.108461,4830.883469
2,218.335616,0.324832,910.186916,2.417528,10.382382,94.085593,5.249952,4.504283,0.334988,0.421357,...,20.0,120.0,0,336.0,20.0,224.0,800,16.773659,124.836263,4858.390078
3,277.901886,0.254335,1145.884560,3.622402,8.282241,137.229613,7.212086,3.251764,0.293604,0.550802,...,20.0,120.0,0,336.0,20.0,224.0,800,9.012157,128.357168,4862.630675
4,273.455547,0.305361,1370.442680,3.441763,8.862728,123.493007,4.754028,4.111263,0.314531,0.522093,...,20.0,120.0,0,336.0,20.0,224.0,800,11.999413,135.246755,4852.753832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32763,273.449653,0.260270,1145.757946,2.614628,10.700671,104.489212,5.947154,4.092908,0.233738,0.448196,...,20.0,120.0,0,336.0,20.0,224.0,800,12.120886,121.764510,4866.114604
32764,277.907881,0.304119,1369.509320,2.434000,10.400068,90.731860,6.114197,4.936055,0.358119,0.626810,...,20.0,120.0,0,336.0,20.0,224.0,800,16.268266,130.202227,4853.529507
32765,218.341541,0.243436,1109.569083,3.619842,8.264705,134.332289,5.451334,3.683882,0.251705,0.497369,...,20.0,120.0,0,336.0,20.0,224.0,800,8.807662,130.968098,4860.224240
32766,239.196018,0.339278,1225.251313,2.804519,7.362138,126.717609,7.059604,3.537223,0.294924,0.432007,...,20.0,120.0,0,336.0,20.0,224.0,800,13.063073,107.981254,4878.955673


In [20]:
# Save the samples to a CSV file
br.samples.to_csv(os.path.join(simulation_out_path,
                               f'input_samples_{n_samples}.csv'),
                                 index=False)


In [21]:
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                   time_setup=TEMPLATE_TIME_SETUP_DICT)

In [ ]:
simulations = br.run_batch(
            n_jobs=9,
            output_path=os.path.join(simulation_out_path, f'raw_simulations_{n_samples}_samples') 
        )

 30%|███       | 9855/32768 [14:14<35:41, 10.70it/s]

In [ ]:
import json

bool_indices = [index for index, value in enumerate(simulations) if isinstance(value, bool)]

with open(os.path.join(simulation_out_path,"bool_indices.json"), 'w') as f:
    json.dump(bool_indices, f)

print(bool_indices)  

[]


In [ ]:
summary_rows = []

for i_sim in range(len(simulations)): ## stuck with the length of simulations like fay
    sim_df = simulations[i_sim]
    
    # Skip if not a DataFrame this was needed because sometimes simulations can return None or other types. or false.
    if not isinstance(sim_df, pd.DataFrame):
        print(f"Simulation {i_sim} is not a DataFrame, skipping.")
        continue

    row = {} ##the 'dictiionary' to hold the summary for each simulation
    for h in sim_df.columns: # We had sim_df the whole time which means h can be used without the headers list!
        row[f"{h}_mean"] = sim_df[h].mean()
        row[f"{h}_max"] = sim_df[h].max() ## using the headers defined above to make the new csv file headers eg v_ao_mean, v_ao_max, etc.
    summary_rows.append(row) # this is a list that is being used to collect summary data for our simulations.
    print(f"Simulation {i_sim} done.") # simple way to track progress 

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(os.path.join(simulation_out_path, "simulations_summary.csv"), index=False)  ## the location, nice and organised!
print("Saved as simulation_summary.csv") # not really needed but nice to show the file has been saved! :D

Simulation 0 done.
Simulation 1 done.
Simulation 2 done.
Simulation 3 done.
Simulation 4 done.
Simulation 5 done.
Simulation 6 done.
Simulation 7 done.
Simulation 8 done.
Simulation 9 done.
Simulation 10 done.
Simulation 11 done.
Simulation 12 done.
Simulation 13 done.
Simulation 14 done.
Simulation 15 done.
Simulation 16 done.
Simulation 17 done.
Simulation 18 done.
Simulation 19 done.
Simulation 20 done.
Simulation 21 done.
Simulation 22 done.
Simulation 23 done.
Simulation 24 done.
Simulation 25 done.
Simulation 26 done.
Simulation 27 done.
Simulation 28 done.
Simulation 29 done.
Simulation 30 done.
Simulation 31 done.
Simulation 32 done.
Simulation 33 done.
Simulation 34 done.
Simulation 35 done.
Simulation 36 done.
Simulation 37 done.
Simulation 38 done.
Simulation 39 done.
Simulation 40 done.
Simulation 41 done.
Simulation 42 done.
Simulation 43 done.
Simulation 44 done.
Simulation 45 done.
Simulation 46 done.
Simulation 47 done.
Simulation 48 done.
Simulation 49 done.
Simulation